Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 1.31 ms, sys: 6.64 ms, total: 7.95 ms
Wall time: 12 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.1.0.7:8787/status,
Dashboard: http://10.1.0.7:8787/status,Workers: 1
Total threads: 6,Total memory: 6.78 GiB
Status: running,Using processes: False
Comm: inproc://10.1.0.7/8267/1,Workers: 1
Dashboard: http://10.1.0.7:8787/status,Total threads: 6
Started: Just now,Total memory: 6.78 GiB
Comm: inproc://10.1.0.7/8267/4,Total threads: 6
Dashboard: http://10.1.0.7:40175/status,Memory: 6.78 GiB
Nanny: None,


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 1.01 s, sys: 98.2 ms, total: 1.11 s
Wall time: 2.71 s


[1.2518391185344768,
 1.967300699173027,
 1.1921768891007014,
 1.1508851343978415,
 1.0869530251966029,
 1.3395127512611615,
 1.8353113216002974,
 1.7138099744257085,
 1.6679971818055845,
 1.8647842720985184,
 1.3210153422733102,
 1.5413927605930233,
 1.164042907553493,
 1.0314580101042854,
 1.0711471685374874,
 1.0442834426380299,
 1.931199989512107,
 1.3644475100733584,
 1.9334433605591859,
 1.9817491037589097,
 1.406801443445758,
 1.8909888768695662,
 1.2837339021441623,
 1.7395819078283443,
 1.999778584542157,
 1.6423609078708132,
 1.396604708570918,
 1.3887852336442459,
 1.5926609545860728,
 1.7886932827376854,
 1.2265532338512402,
 1.9138232304572873,
 1.1197234843696666,
 1.134131012517082,
 1.5875756875849345,
 1.6946853986324157,
 1.7590876487763478,
 1.2440401658236415,
 1.4094107347747094,
 1.6743308715242575,
 1.0874906530079336,
 1.3001027366763596,
 1.7952312675387587,
 1.3753142079981628,
 1.9816492104338934,
 1.1165130167276827,
 1.5947212531479302,
 1.5123508773551202,

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.1379214803897966,
 1.8004468997619592,
 1.1477710117536066,
 1.1722491087770073,
 1.2182080829905781,
 1.1305521329516894,
 1.5315118828105008,
 1.4857109550716339,
 1.1564552302047688,
 1.7632671735712062,
 1.7043367478356086,
 1.6531882978486887,
 1.6449568037500053,
 1.1441160009442368,
 1.8717558171360351,
 1.3583440002967633,
 1.693922800784076,
 1.8780095173736684,
 1.9653715196710575,
 1.5787513479684643,
 1.0882561072536192,
 1.4834508964954152,
 1.9824283555788558,
 1.7626012062972949,
 1.9614876283293154,
 1.5538150550125693,
 1.0835788240162896,
 1.1465480726253297,
 1.1279461112962195,
 1.1496712116177383,
 1.1477319237177976,
 1.1999783254506513,
 1.217967638377778,
 1.0951703079673132,
 1.0611877381374426,
 1.1880676743279521,
 1.0103101046672116,
 1.3912816099354766]

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 310 ms, sys: 63.4 ms, total: 373 ms
Wall time: 2.54 s
